# Real Estate Rental Market in Berlin. 
# p.1.  Parsing. Cleaning. Processing.

I was inspired by original ideas and some useful approaches that were taken from [Dmitrii Eliuseev](https://towardsdatascience.com/housing-rental-market-in-germany-exploratory-data-analysis-with-python-3975428d07d2).

This notebook is an attempt to experiment with approaches that I found very useful and interesting, and they have their origins in the TDS article 'Housing Rental Market in Germany: Exploratory Data Analysis with Python'.  
The scope and processing are widen greatly in order to collect as much data as possible.

I will try to find some trends and insights from the data collected on https://www.immobilienscout24.de as one of the largest online residential rental aggregators in Germany.  

The main stages of the forthcoming work:  
|part| Main tages|Desciption|
|:---|:---|:---|
|p1.| Ask| goals of the research|
||Prepare| parcing the site, collecting data, basic cleaning|
||Proccess| cleaning w.r.t. context, transforming data, conducting feature engineering|
|p2.| Analyze| analizing, building up a simple regression model for predicting the prices|
|| Share| main takeaways and some visualizations|

Loading the environment.  
You need to uncomment some lines of code if these libraries are not installed on your system. 

In [94]:
import os
import pandas as pd
import numpy as np
#pip install selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import plotly.express as px

import json
import re #regular expression
# pip install googletrans==4.0.0-rc1
from googletrans import Translator

import folium
from geopy.geocoders import Nominatim
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


Defining some variables to configure the proccess.

In [95]:
to_parce, to_translate = False, False
base_url = "https://www.immobilienscout24.de"
path_to_csv = "/Users/velo1/SynologyDrive/GIT_syno/data/immobilienscout24.de/"
cols =  'property_id, title, logging_date, property_area, num_rooms, num_bedrooms, num_bathrooms, criteria, garage, floor, floors_in_building, constr_year, energy_eff, add_costs, heat_costs, cold_price, warm_price, deposit, property_type, publisher, contact, city, address, description, region, zip, link'.split(', ')

# pd.set_option('display.max_colwidth', 100) # to display full text in columns
pd.set_option('display.max_columns', None) # display all columns

## Ask

1. What is the most popular residential rental objects in Berlin?  
1. What are the main factors that define the rental price?  
1. Are there any trends and hidden patterns?
1. What are the main segments of that rental market?

## Prepare

|instance| used for storing:|
|:---|:---|
|base_url |https://www.immobilienscout24.de|
|to_parce, to_translate|boolean flags to run parcing the site, translating some fields to English|
|||
|Berlin_housing.csv|raw data with basic proccessing|
|Berlin_housing_eng.csv|partitially proccessed and translated data |
|||
|df_raw |input data  with basic proccessing|
|df |cleaned data|



### Data collecting. Parsing 

For this particular notebook I use [immobilienscout24.de](https://www.immobilienscout24.de) as one of the most popular site on local german market.  



#### Parcing with `Requests` library.

In [96]:
import requests

base_url = "https://www.immobilienscout24.de"
url_berlin = base_url + "/Suche/de/berlin/berlin/wohnung-mieten" 
print(requests.get(url_berlin))

<Response [403]>


The server returns <Response [403]>.  
It seems that the page rejects GET requests that do not identify a User-Agent.  
This approach doesn't work with this particular site but we've made a try.


Let's try a Selenium approach which takes under control a Chrome browser and emulate a real user browsing. 

#### Parcing with the `Selenium` python library 
allows using a real Chrome browser to retrieve the data and automate reading pages.  
Parcing this page was a real challenge for me.  
I blocked the loading of images, experimented with the delay and finally got the result.
Here are some functions to control the parsing process:

There are prices in listings given in a few opposite formats like:  
|1.000,00|1,000.00 EUR|1.000,00|  
|:---|:---|:---|
|We'll fix all such cases with `fix_numeric_inconstancies()` function:  
|1000.00|1000.00|1000.00|

In [97]:
# regex pattern to extract numbers interspersed with commas and dots from strings
fix_numeric_inconstancies_regex_pattern = re.compile(r'\D*([\d+[\,\.]*\d*]*)') 

def fix_numeric_inconstancies(string):
    '''Fixes inconsistencies in price strings 
    e.g. 1.000,00, 1.000,00 EUR, 1.000,00 EUR/m, 1.000,00 EUR/m
    '''    
    if isinstance(string, str):
        try:
            res = fix_numeric_inconstancies_regex_pattern.search(string).group(1)
            res = res.replace(',', '.')            # replace commas with dots
            res = re.sub(r'\.(?=.*\.)', '', res)   # delete all dots except the LAST one
            if res == '':                          # if the string is empty, return 0
                return np.nan
            else:
                num =len(re.split(r'\.',res)[-1] )   # split the string into parts: before and after the dot
                if num == 3:                         # if the number of digits after the LAST dot is more than 2
                    if len(re.split(r'\.',res)[0] ) < 3: 
                        return 10**num*float(res)       # return 10**num*float(res)                             
            return float(res)
        

        except AttributeError:
            pass
    return string

In [98]:
fix_numeric_inconstancies('€1.900,00'),fix_numeric_inconstancies('€1,900.00'),\
  fix_numeric_inconstancies('Sofort€1,900.00keine'),fix_numeric_inconstancies('€9.9.01,00/m')

(1900.0, 1900.0, 1900.0, 9901.0)

In [99]:
fix_numeric_inconstancies('€  100,00'),fix_numeric_inconstancies('€  20,000'),\
  fix_numeric_inconstancies('€  10.000'), fix_numeric_inconstancies('.')

(100.0, 20000.0, 10000.0, nan)

In [100]:
def page_has_loaded(driver: webdriver.Chrome): 
    """ Check if the page is ready """
    page_state = driver.execute_script('return document.readyState;') 
    return page_state == 'complete'


def page_get(url: str, driver: webdriver.Chrome, delay_sec: int):
    """ Get the page content """
    driver.get(url)                     # load the page
    time.sleep(delay_sec)               # wait for the page to load
    while not page_has_loaded(driver):  # wait until the page is loaded (page_state == 'complete')
        time.sleep(0.1)
    return driver.page_source           # return the page content

def get_links(html: str, pp= 0):   
    ''' Retrive the links to the subpages from the main search pages results'''
    soup = BeautifulSoup(html, "lxml")          # parse the html using beautiful soup and store in variable `soup`
    li = soup.find(id="resultListItems")        # where the sublinks are stored
    links_all = []                              # list of links

    children = li.find_all("li", {"class": "result-list__listing"}) # this instance stores the links to the subpages
    for child in children:
        for link in child.find_all("a"):
            if 'data-go-to-expose-id' in link.attrs:                # check if the link has the required attributes
                links_all.append(base_url + link['href'])
                break

    links_all.append(base_url + link['href'])
    
    print(f'Got {len(links_all)} links, page:{pp} ')# print the number of links found and the page number
                                                    # in a case of an error, the page number can be used to restart the parcing loop
                                                    # from the last page that was successfully parsed
    os.system(f'say Got {len(links_all)} links, page:{pp} ')
    return links_all

def get_attributes(soup, link = None):  
    """ 
    Get the attributes of the property from the soup object
    """

    # initialize the empty variables
    str_property_id, str_logging_date, str_property_area, str_num_rooms, str_num_bedrooms, str_num_bathrooms, str_criteria, str_garage, str_floor, str_floors_in_building, str_year, str_energy_efficiency, str_add_costs, str_energy_costs, str_cold_price, str_warm_price, str_deposit, str_property_type, str_publisher, str_contact, str_city, str_title, str_address, str_desciption, str_region, str_zip = \
    ('',)*26

    # get the attributes from the soup object
    property_id = soup.find_all("div", "is24-scoutid__content") 
    if len(property_id) > 0:
        str_property_id = property_id[0].get_text().strip().split("Scout-ID: ")[1]
    
    logging_date = soup.find_all("dd", "is24qa-bezugsfrei-ab grid-item three-fifths")
    if len(logging_date) > 0:
        str_logging_date = logging_date[0].get_text().strip()
        
    property_area = soup.find_all("div", "is24qa-flaeche-main is24-value font-semibold")
    if len(property_area) > 0:
        str_property_area = property_area[0].get_text().strip()
        str_property_area = str_property_area.replace("m²", "").replace(".", "").strip()

    num_rooms = soup.find_all('dd', "is24qa-zimmer")
    if len(num_rooms) > 0:
        str_num_rooms = num_rooms[0].get_text().strip()
    
    num_bedrooms = soup.find_all("dd", "is24qa-schlafzimmer")
    if len(num_bedrooms) > 0:
        str_num_bedrooms = num_bedrooms[0].get_text().strip()
    
    num_bathrooms = soup.find_all("dd", "is24qa-badezimmer")
    if len(num_bathrooms) > 0:
        str_num_bathrooms = num_bathrooms[0].get_text().strip()

    criteria = soup.find_all("div", "criteriagroup boolean-listing padding-top-l")
    if len(criteria) > 0:
        str_criteria = criteria[0].get_text().replace('\n',' ').strip()
    
    garage = soup.find_all("dd", "is24qa-garage-stellplatz")
    if len(garage) > 0:
        str_garage = garage[0].get_text().strip()

    floor = soup.find_all("dd", "is24qa-etage")
    if len(floor) > 0:              # check if the floor is available
        temp_floor = floor[0].get_text().strip().split("von")
        str_floor = temp_floor[0].strip()
        if len(temp_floor) > 1:     # check if the number of floors is available
            str_floors_in_building = temp_floor[1].strip()

    year =soup.find_all("dd", "is24qa-baujahr")
    if len(year) > 0:
        str_year = year[0].get_text().strip()

    energy_efficiency = soup.find_all("dd", "is24qa-energieeffizienzklasse")
    if len(energy_efficiency) > 0:
        str_energy_efficiency = energy_efficiency[0].get_text().strip()

    energy_costs = soup.find_all("dd", "is24qa-heizkosten grid-item three-fifths")
    if len(energy_costs) > 0:
        str_energy_costs = energy_costs[0].get_text().replace('+','').replace('€','').strip()

    add_costs = soup.find_all("dd", "is24qa-nebenkosten")
    if len(add_costs) > 0:
        str_add_costs = add_costs[0].get_text().replace('+','').replace('€','').strip()        
    

    cold_price = soup.find_all("div", "is24qa-kaltmiete-main")
    if len(cold_price) > 0:
        str_cold_price= cold_price[0].get_text().strip()
        # Your locale maybe different from immobilienscout24.  
        # In this case your should make a little changes to regex patterns I used.  
        # Site locale    `2.000,00 €`,  my system locale  `2000.00`        
        str_cold_price = str(fix_numeric_inconstancies(str_cold_price))

    warm_price = soup.find_all("div", "is24qa-warmmiete-main")
    if len(warm_price) > 0:
        str_warm_price = warm_price[0].get_text().strip()  
        str_warm_price = str(fix_numeric_inconstancies(str_warm_price))

    deposit = soup.find_all("div", "is24qa-kaution-o-genossenschaftsanteile")
    if len(deposit) > 0:
        str_deposit = deposit[0].get_text().strip()

    property_type = soup.find_all("dd", "is24qa-typ grid-item three-fifths")
    if len(property_type) > 0:
        str_property_type = property_type[0].get_text().strip()
    
    publisher = soup.find_all(attrs={"data-qa": "company-name"})  #  , "companyName"
    if len(publisher) > 0:
        str_publisher = publisher[0].get_text().strip()
    else:
        item = soup.find("div", {"class": "brandLogoPrivate_dnns4"})
        if item is not None:
            str_publisher = "Private"      

    contact = soup.find_all(attrs={"data-qa": "contactName"})
    if len(contact) > 0:
        str_contact = contact[0].get_text()


    title = soup.find_all("h1", id="expose-title")
    if len(title) > 0:
        str_title = title[0].get_text().strip()

    desciption = soup.find_all("pre", "is24qa-objektbeschreibung")
    if len(desciption) > 0:
        str_desciption = desciption[0].get_text().replace('\n', ' ').replace(';', ',').strip()

    region = soup.find_all("span", "zip-region-and-country")
    if len(region) > 0:
        str_region = region[0].get_text().strip().split(",")[0].strip()
        str_city = region[0].get_text().strip().split(",")[1].strip()
        str_city = str_city.split(" ")
        str_zip, str_city = str_city[0], str_city[1]

    address = soup.find_all("span", "block font-nowrap print-hide")
    if len(address) > 0:
        str_address = address[0].get_text().strip()

    return [str_property_id, str_title, str_logging_date, str_property_area, str_num_rooms, str_num_bedrooms, str_num_bathrooms, str_criteria, str_garage, str_floor, str_floors_in_building, str_year, str_energy_efficiency, str_add_costs, str_energy_costs, str_cold_price, str_warm_price, str_deposit, str_property_type, str_publisher, str_contact, str_city,  str_address, str_desciption, str_region, str_zip, link]


This chunk of code was intended to automatically prevent image loading and increase the performance of parcing.  
But the site has a sophisticated antirobot checkings that require reviewing images in order to pass a test, so this approach didn't work.  
We still need to manually turn off image loading.  
However, there will be sound and printed warnings.

In [101]:
# Block images via ChromeOptions object
# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.managed_default_content_settings.images": 2}
# chrome_options.add_experimental_option("prefs", prefs)

##### The Parsing.

In [102]:
if to_parce:
    
    # To continue parsing after an error, 
    # set the start_page to the page you want to start parsing
    start_page = int(input(f'What page in search pages do you want to start parsing?'))
    depth = int(input(f'How many pages do you want to parse?'))


    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options= chrome_options)
    print("Driver is ready. \nYou have 60s to DISABLE images loading ...\nPrivacy and Security -> Site Settings -> Images -> Don't allow site to show images\n")
    os.system(f'say "Driver is ready. You have 60 seconds to DISABLE images loading"')
 
    cnt = 0
    for pp in range(start_page, start_page+ depth + 1):

        if pp == 1:

            # open the file in the write mode and write the header row  with the column names (OVERWRITE THE FILE)
            with open(path_to_csv + 'Berlin_housing.csv', 'w') as f:  # write header row

                f.write("; ".join(cols)+'\n')

        if cnt == 0:        # first page
            delay_sec = 60  # wait 60 sec to have time to login , accept cookies and block images from loading
            cnt += 1

        else:
            delay_sec = np.random.random()*0.5 # wait random time to avoid bot detection

        if pp == 1: # first page
            url_page = base_url + "/Suche/de/berlin/berlin/wohnung-mieten"

        else:       # other pages
            url_page = base_url +  "/Suche/de/berlin/berlin/wohnung-mieten?pagenumber=" + str(pp)


        html = page_get(url_page, driver, delay_sec= delay_sec) # go to search page
        links_all = get_links(html, pp)                         # get links from search page

        for link in links_all:                                  # go to each link

            s_html = page_get(link, driver, delay_sec= np.random.random()*0.5)
            soup = BeautifulSoup(s_html, "lxml")
            row = get_attributes(soup, link)                          # get attributes from each link

            with open(path_to_csv + 'Berlin_housing.csv', 'a') as f:  # write to csv file    
                f.write(";".join(row)+'\n')

    driver.close()

##### The results of parsing the site are stored in 'Berlin_housing.csv'

In [103]:
# df_raw.to_csv(path_to_csv + 'Berlin_housing.csv', sep=';', index=False)  

##### Loading tha data we've already parced.  
This is useful if you've finished parcing and continue the next stages of research later.

In [104]:
df_raw = pd.read_csv(path_to_csv + 'Berlin_housing.csv',  names= cols, header=0,  sep=';', on_bad_lines='skip') 
df_raw.head(3)

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link
0,141131393,Nassauische Straße! Helle 6-Zimmer-Altbau-Wohn...,sofort bzw. nach Vereinbarung,"220,5",7,3,NaN,Balkon/ Terrasse Balkon/ Terrasse Keller Kelle...,NaN,1,5,1900,NaN,800,in Nebenkosten enthalten,3500,4300,3 Nettokaltmieten,Etagenwohnung,Kupsch Wohnimmobilien GmbH,Frau Sabine Woide Immobilien,Berlin,NaN,Berlin- Wilmersdorf Wohnquartier Güntzelkiez (...,Wilmersdorf,10717,https://www.immobilienscout24.de/expose/141131393
1,141131071,Tauschwohnung: Schöne 2-Zi im Gräfekiez gegen ...,NaN,60,2,NaN,NaN,Einbauküche Einbauküche,NaN,3,NaN,NaN,NaN,170,keine Angabe,410,580,NaN,NaN,Tauschwohnung GmbH,Tauschwohnung Wohnungstausch,Berlin,NaN,Ruhige und schöne Wohnung im Gräfekiez. Ideal ...,Kreuzberg,10967,https://www.immobilienscout24.de/expose/141131071
2,141159056,"Tauschwohnung: Schöne 2-Zi Whg in PB, 3-4 Zi-W...",NaN,54,2,NaN,NaN,Keller Keller,NaN,1,NaN,NaN,NaN,127,keine Angabe,456,583,NaN,Etagenwohnung,Tauschwohnung GmbH,Tauschwohnung Wohnungstausch,Berlin,NaN,"Hallo, unsere kleine Familie (2 Erwachsene und...",Prenzlauer Berg,10407,https://www.immobilienscout24.de/expose/141159056


### Basic data cleaning  
#### Duplicates

In [105]:
print(df_raw.duplicated().sum())  #.any()
df_raw.drop_duplicates(inplace=True)

294


#### Nan values
Some nans we drop right now,  
others might be droped or filled later taking a context into account.

In [106]:
def check_na(df, sort = 'dtype'):
  '''
  Check for missing values in a dataframe
  df - dataframe
  sort - sort by column name or by dtype or by nans% (if `category` dtype is present)
  '''
  sort = ['dtype', 'nans%'] if sort  else ['nans%']
  dict_ = {}
  for col in df.columns:
    dict_[col] = {'dtype':df[col].dtype, 'nans':df[col].isna().sum(), 'nans%':df[col].isna().sum()/df.shape[0]*100}
  return pd.DataFrame(dict_).T.sort_values(by=sort, ascending=False)\
                    .style.bar(subset=['nans%'], color='#faebd7').format(precision=1, thousands=",")

In [107]:
check_na(df_raw)

,dtype,nans,nans%
garage,object,"3,844",88.9
energy_eff,object,"3,773",87.3
num_bedrooms,object,"3,233",74.8
floors_in_building,object,"3,201",74.1
constr_year,object,"3,139",72.6
num_bathrooms,object,"3,099",71.7
logging_date,object,"2,986",69.1
property_type,object,"2,985",69.1
floor,object,"2,220",51.4
criteria,object,"1,913",44.3


We see a significant number of missing values in all attributes.  
 
Partly this is due to the abscence of strict data input formatting on the site itself.

Now let's drop rows without essentaial attributes such as  
`property_id`, `cold_price` or `link`.   
The abscence of this information is might be a result of parcing errors. 

In [108]:
ind = df_raw[(df_raw['property_id'].isna() )| (df_raw['cold_price'].isna() )| (df_raw.link.isna())].index
df_raw.drop(ind, inplace=True)

#### Copying partially cleaned data to a new instance.
The most obvious preparations have been done.  
Now we are copying the data to a new instance for processing.

In [109]:
df = df_raw.copy()

## Process
It's a technical part with explanations of data processing I made.

### Let's translate some attributes to English.

In [110]:
# service code demonstrating how to translate german words to english
translator = Translator()
translator.translate("in Nebenkosten enthalten", dest='en', src='german').text

'included in additional costs'

#### garage
This attribute is the least represented in the data (90 % of nans).  
What types of garages are offered?

In [111]:
df.garage.unique()

array([nan, '1 Tiefgaragen-Stellplatz', 'Tiefgaragen-Stellplatz',
       '98 Tiefgaragen-Stellplätze', '1 Außenstellplatz',
       'Außenstellplatz', '1 Stellplatz', '1 Duplex-Stellplatz',
       'Parkhaus-Stellplatz', '3 Tiefgaragen-Stellplätze', '1 Garage',
       'Garage', '2 Tiefgaragen-Stellplätze', '2 Außenstellplätze',
       '2 Stellplätze', '1 Carport', '16 Tiefgaragen-Stellplätze',
       '4 Außenstellplätze', '2 Garagen'], dtype=object)

Translate garage types to English.

In [112]:
dict_ = {'Außenstellplatz':'Outdoor parking space', 'Tiefgaragen-Stellplatz':'Underground parking space',
'Tiefgaragen-Stellplätze':'Underground parking spaces', 'Tiefgarage':'Underground garage', 'Außenstellplätze':'Outdoor parking spaces',
'Garage':'garage', 'Stellplatz':'parking space','Parkhaus-Stellplatz':'Parking garage parking space',
'garagen':'garages', 'Parkhaus':'Parking garage','Stellplätze':'parking spaces', 'Garagen':'garages',
'Carport':'Carport', 'Duplex-Stellplatz':'Duplex parking space', 'Parkplatz':'Parking space'}

df.garage = df.garage.replace(dict_, regex=True)
df.garage.unique()

array([nan, '1 Underground parking space', 'Underground parking space',
       '98 Underground parking spaces', '1 Outdoor parking space',
       'Outdoor parking space', '1 parking space',
       '1 Duplex-parking space', 'Parking garage-parking space',
       '3 Underground parking spaces', '1 garage', 'garage',
       '2 Underground parking spaces', '2 Outdoor parking spaces',
       '2 parking spaces', '1 Carport', '16 Underground parking spaces',
       '4 Outdoor parking spaces', '2 garagen'], dtype=object)

This information includes only the possibility of using a garage or a parking place.  
Prices do not include using a garage.

#### property_type

In [113]:
df.property_type.unique()
dict_ = {'Dachgeschoss':'Attic', 'Erdgeschosswohnung':'Ground floor apartment',
'Hochparterre':'High parterre', 'Etagenwohnung':'Flat', 'Souterrain':'Basement',
'Terrassenwohnung':'Terrace apartment', 'Sonstige':'Other', 'Maisonette':'Small house',}

df.property_type = df.property_type.replace(dict_, regex=True)
df.property_type.unique()

array(['Flat', nan, 'Small house', 'Ground floor apartment', 'Attic',
       'Penthouse', 'Basement', 'Terrace apartment', 'Loft',
       'High parterre', 'Other'], dtype=object)

#### logging_date
This is not a required attribute.  
70 % of owners doesn't mention logging date at all.  
But some of them make comments, and we will translate them into English.

In [114]:
dict_ = {'nach Absprache':'according to the arrangement', 'sofort':'immediately','Sofort':'Immediately','verfügbar':'accessible',
'Mietbeginn':'Start of rental','Nach Vereinbarung':'By appointment','bzw.': 'or','nach':'after','Fertigstellung':'completion',
'bezugsfrei':'free of charge','Vereinbarung':'agreement','ab':'from','bis':'to','ab sofort':'immediately',
'voraussichtlich':'probably','Voraussichtlich':'Probably','Voraussichtlich':'Probably','Voraussichtlich':'Probably',
'Sommer':'summer','Winter':'winter','Frühjahr':'spring','Herbst':'autumn','Ende':'end','Anfang':'beginning',
'mitte': 'middle', 'Mitte':'Middle','kurzfristig':'short term','Kurzfristig':'Short term','Kurzfristig':'Short term', 'Nach Absprache':'according to the arrangement'}
df.logging_date = df.logging_date.replace(dict_, regex=True)

#### const_year   
72 % of listings do not mention a construction year.  
Some listings define `const_year` as 'unbekannt'.
Let's translate.  
Everything is clear here

In [115]:
df.constr_year = df.constr_year.replace({'unbekannt': 'Unknonw'}, regex=True)

####  add_costs,  heat_costs
Some owners make a notes in a free form.  
If 'in Nebenkosten enthalten' ('included in additional costs') then we mark the costs as '0'.  
We will process them more carefully later.

In [116]:
dict_ = {'in Nebenkosten enthalten':'0', 'enthalten':'contain','nicht':'not','keine Angabe':'not specified'}
df.heat_costs = df.heat_costs.replace(dict_, regex=True)
df.add_costs= df.add_costs.replace(dict_, regex=True)

#### title, criteria, description
Here I have a real challange as I haven't api keys for batch translating (and significantly increase the speed).  
The following proccess is executed row by row with online requests to Google.  
There were many timeouts and other issues so I divide the translation into chunks.  

In [117]:
def translate_col(df, columns, chunk_size=300, start_chunk_num=1):
  '''
  Translate column in dataframe
  df - dataframe
  columns - list of columns to translate
  chunk_size - number of rows to translate at once
  start_chunk_num - number of chunk to start from
  '''

  error_chunk = 0

  for ch in range(start_chunk_num, df.shape[0]//chunk_size + 2):
    print(f'Chunk {ch} of {df.shape[0]//chunk_size }')
    # os.system(f'say Chunk {str(ch)} started.')

    ind1 = ch * chunk_size - chunk_size
    ind2 = ch * chunk_size if ch * chunk_size < df.shape[0] else df.shape[0]

    print(f'ind1 {ind1}, ind2 {ind2}', end=' ')

    for col in columns:
      try:
        df.loc[ind1:ind2, col] = df.loc[ind1:ind2, col].apply(lambda x: translator.translate(x, dest='en', src='auto').text)
      except:
        print(f'Error in column {col} at index {ind1} - {ind2}')
        os.system(f'say Error in column {col} at index {ind1} - {ind2}')
        error_chunk = ch 
        return error_chunk  # error
      
      time.sleep(.1) 
      print(translator.translate("Everything's under control", dest='german', src='auto').text +':' , end=' '   )
      print(f'Column: {col} translated.', end=' ')
      os.system(f'say Column {col} translated')
      

    time.sleep(.1) 
    print()
  return  0   # no error

`The next chunk of code maybe running for a long time.`  

Loops will repeat and repeat  until the successful executition without errors will be performed.  
You can skip this stage and load the intermediate results.

In [118]:
if to_translate:
  error_chunk = 727    # start from this chunk if you continue after a break, otherwise 1
  # this loop will continue until all chunks are translated 100% without errors
  while True:
    error_chunk = translate_col(df, ['title', 'description', 'criteria'], chunk_size=5, start_chunk_num= error_chunk)
    # higher chunk sizes increase the speed of translation but also increase the chance of errors 
    if error_chunk == 0:  # 0 - no error  (change to 0 if you want to run all chunks)
                          # or > 0 to limit the number of chunks)
      break

  os.system('say "Beer time"')

#### Saving the intermediate results of translation from German to English.

In [119]:
# df.to_csv(path_to_csv + 'Berlin_housing_ENG2.csv', sep=';', index=False)

### Nan values, data types

At this stage we'll define
* a proper data types 
* fill some features based on their context
* check for possible outliers

#### Loading intermediate results with some columns have already been translated.

In [120]:
df= pd.read_csv(path_to_csv + 'Berlin_housing_ENG2.csv', names= cols,  header=0,  sep=';', \
                dtype= {'floor':str,'floors_in_building':str,'num_bathrooms':str,'num_bedrooms':str}) 
df.head(2)

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link
0,141131393,Nassauische Straße!Bright 6-room alcohol apart...,immediately or after agreement,"220,5",7,3,NaN,Balcony/ terrace balcony/ terrace basement bas...,NaN,1,5,1900.0,NaN,800,included in additional costs,3500,4300,3 Nettokaltmieten,Flat,Kupsch Wohnimmobilien GmbH,Frau Sabine Woide Immobilien,Berlin,NaN,Berlin-Wilmersdorf Wohnquartier Güntzelkiez (T...,Wilmersdorf,10717,https://www.immobilienscout24.de/expose/141131393
1,141131071,Exchange apartment: beautiful 2-room in Gräfek...,NaN,60,2,NaN,NaN,Fitted kitchen fitted kitchen,NaN,3,NaN,NaN,NaN,170,not specified,410,580,NaN,NaN,Tauschwohnung GmbH,Tauschwohnung Wohnungstausch,Berlin,NaN,Quiet and beautiful apartment in the Gräfekiez...,Kreuzberg,10967,https://www.immobilienscout24.de/expose/141131071


In [31]:
check_na(df)

,dtype,nans,nans%
garage,object,"3,696",90.7
energy_eff,object,"3,646",89.5
num_bedrooms,object,"3,140",77.1
floors_in_building,object,"3,099",76.1
num_bathrooms,object,"3,016",74.0
logging_date,object,"2,908",71.4
property_type,object,"2,739",67.2
floor,object,"2,139",52.5
deposit,object,"1,440",35.4
address,object,"1,001",24.6


#### `property_area`

In [89]:
df.property_area = df.property_area.apply(fix_numeric_inconstancies).astype('float16')


Let's take a closer look at property sizes.

In [90]:
fig = px.box(df[['property_area']], x= 'property_area', notched=True, title='property_area')
fig.update_layout(xaxis_title="Property size (sq.meters)", yaxis_title="")

In [91]:
df[df.property_area>300]

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs,deposit_calc,year_group,criteria_clean
305,140099183,Life in the Monbijou residence - stately penth...,,375.0,8.0,4,4,Balcony/ terrace balcony/ terrace basement bas...,1 parking space,?,?,1906.0,D,1.724,included in additional costs,9850.000000,11574.000000,"29.403,00 EUR",Other,Engel & Völkers Berlin Mitte GmbH,Engel & Völkers Berlin Mitte,Berlin,"Monbijoustraße 3/5,",This stately maisonett apartment has eight roo...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/140099183,0.0,1724.00,0.0,26.266666,1724.000000,29403.000000,First half XX cent,balcony terrace basement passenger elevator pe...
311,139142000,Berlin in view - unique townhouse in the heart...,from immediately,456.0,4.0,3,3,Balcony/ Terrace Balcony/ Terrace Passenger Ri...,1 garage,?,?,2012.0,Unknown,1.284,not included in additional costs,15000.000000,16284.000000,"45.000,00 EUR",Other,Engel & Völkers Berlin Mitte GmbH,Engel & Völkers Berlin Mitte,Berlin,"Oberwallstraße 13,",Characteristic of the town houses are the long...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/139142000,NaN,1284.00,NaN,32.894737,1284.000000,45000.000000,modern,balcony terrace passenger kitchen guest toilet
315,141306965,Exclusive townhouse in Mitte near the Gendarme...,immediately,456.0,4.0,3,3,Balcony/ terrace balcony/ terrace basement bas...,1 Underground parking space,?,?,2012.0,Unknown,1.284,not specified,15000.000000,16284.000000,3 NKM,Small house,FAMOZA Immobilien,Frau Josipa Kovačević,Berlin,,An exclusive townhouse in the popular Mitte di...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/141306965,NaN,1284.00,NaN,32.894737,1284.000000,45000.000000,modern,balcony terrace basement fitted kitchen built-...
1886,140875809,First cover: spectacular penthouse in city loc...,01.04.2023,321.0,5.0,3,2,Balcony/ terrace balcony/ terrace basement bas...,2 Underground parking spaces,6,6,2022.0,Unknown,950,included in additional costs,11500.000000,12450.000000,0,Penthouse,Engel & Völkers Immobilien Deutschland GmbH,Engel & Völkers Immobilien Deutschland GmbH,Berlin,,The penthouse offered here is located on the K...,Charlottenburg,10625,https://www.immobilienscout24.de/expose/140875809,0.0,950.00,0.0,35.825546,950.000000,0.000000,new,balcony terrace basement passenger elevator pe...
1908,138488871,Exceptional Living in Jägerstraße on Friedrich...,Nach Absprache,706.0,7.0,4,3,Balcony/ terrace balcony/ terrace basement bas...,Underground parking space,6,6,2007.0,C,2.850,included in additional costs,17000.000000,19850.000000,0,Penthouse,CITY-CONCEPT Gesellschaft für Immobilienmanage...,Herr Stefan Schepers,Berlin,"Jägerstraße 34,",The Jägerstraße 34/35 residential and commerci...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/138488871,0.0,2850.00,0.0,24.079321,2850.000000,0.000000,2000-2014,balcony terrace basement passenger elevator pe...
4049,105850244,5-room apartment with a large terrace in the h...,from Juli 2023,343.5,5.0,4,4,Balcony/ terrace balcony/ terrace Passenger el...,1 Underground parking space,3,7,2015.0,Unknown,"1.545,35",included in additional costs,7081.870117,8627.219727,3 Nettokaltmieten,Terrace apartment,HGHI Immobilien Verwaltung GmbH,Frau Marie-Josephine Wahn,Berlin,"Leipziger Str. 12,",As a unique residential area over the roofs of...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/105850244,0.0,1545.35,0.0,20.616798,1545.349609,21245.610352,modern,balcony terrace passenger elevator personal ki...


These are very special offers with a crazy price of up to 20,000 euros for a 706 sq.m penthouse.  
You have to be very wealthy to afford that.  
How can one not remember the film `Scent of a Woman` with Al Pacino in the title role.  
Although it seems to me that the colonel would not allow himself such;)  

But in other way, the relative prices per sq.m. are decent (appr. 25 eur) and  
a lot cheaper than we've reviewed previously for some of the one-bedroom offerings.

#### `cold_price`

Prices are given in different locales (for punctual Germans - this is not typical;)  
The code is tuned to '1234567.89'

In [35]:
df['cold_price'] = df.cold_price.apply(fix_numeric_inconstancies).astype('float32')

#### `warm_price`

In [ ]:
df['warm_price'] = df.warm_price.apply(fix_numeric_inconstancies).astype('float32')

In [ ]:
fig = px.box(df[['property_area','cold_price','warm_price','heat_costs_calc','add_costs_calc']], notched=True,  boxmode="overlay",
             title='Checking the outliers', height=800, color='variable')
fig.update_xaxes(tickangle=60)
fig.update_yaxes(type="log")
fig.update_layout(xaxis_title="", yaxis_title="Value range (log scale)")
fig.update_traces( showlegend= False)

Prices do definetely have outliers.  
Property area also needs to be checked. 

#### Cold price
This is a pure price without any additional costs.

We have an outlier in the dataset.  
Let's add a `relative cold price` column to explore the prices more intuitively.

In [ ]:
df['cold_price_rel'] = df.cold_price / df.property_area

px.box(df, x='cold_price_rel', height= 300, title='Cold price per square meter')

In [ ]:
df[df.cold_price_rel> 350]    # print out the outliers

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel
2305,140100741,"Wilhelminenhofstraße, Berlin",,68.0,1.0,NaN,NaN,Into,No garage,?,?,NaN,Unknown,not specified,not specified,28000.0,28000.0,1000,Unknown,HousingAnywhere B.V.,,Berlin,"Wilhelminenhofstraße 0,",This apartment offers the privacy of your own ...,Oberschöneweide,12459,https://www.immobilienscout24.de/expose/140100741,NaN,NaN,NaN,411.764709
3575,114866641,"Stylish 1-room apartment in Friedrichshain, Be...",,1.0,1.0,NaN,NaN,Into,No garage,?,?,NaN,Unknown,not specified,included in additional costs,300000.0,300000.0,NaN,Attic,DevCom Deutschland,Herr Test-Vorname Test-Nachname,Berlin,,The apartment includes a pretty room.,Friedrichshain,10243,https://www.immobilienscout24.de/expose/114866641,0.0,NaN,0.0,300000.000000


These ads are most likely have  mistakes as the prices are unreasonably high:  
* 300,000 for 1 sq.m 
* and 28,000 for 68 sq.m  

Let's drop them.

In [ ]:
df.drop(index = df[df.cold_price_rel> 350].index, inplace=True) 

fig = px.box(df[['cold_price_rel']], x='cold_price_rel', notched=True, title='Cold RELATIVE prices <br><sup>€ for sq.m per month (outliers removed)</sup>')
fig.update_layout(xaxis_title="€ for sq.m per month", yaxis_title="Value range")

These results seems to be more realistic with median 22,15 eur for sq.m monthly.  
But prices over 100 euros for square meter per month seems very high.  
Let's explore.

In [ ]:
df[df.cold_price_rel > 150].sort_values(by='cold_price_rel', ascending=False).head(5)

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel
2994,138332662,"Luise Henriette-Straße, Berlin",,19.0,1.0,NaN,NaN,Into,No garage,?,?,NaN,Unknown,not specified,not specified,3528.0,3528.0,0,Unknown,HousingAnywhere B.V.,,Berlin,"Luise-Henriette-Straße 0,",Our 19-23 sqm Suites for stays over 28 nights ...,Tempelhof,12103,https://www.immobilienscout24.de/expose/138332662,NaN,NaN,NaN,185.684204
3288,138325908,"English street, Berlin",,30.0,1.0,NaN,NaN,Into,No garage,?,?,NaN,Unknown,not specified,not specified,5550.0,5550.0,0,Unknown,HousingAnywhere B.V.,,Berlin,"Englische Straße 0,",The essential in perfection. Clear design and ...,Charlottenburg,10587,https://www.immobilienscout24.de/expose/138325908,NaN,NaN,NaN,185.000000
3521,138320888,"Winterfeldtstraße, Berlin",,30.0,1.0,NaN,NaN,Into,No garage,?,?,NaN,Unknown,not specified,not specified,5490.0,5490.0,1200,Unknown,HousingAnywhere B.V.,,Berlin,"Winterfeldtstraße 0,","Comfortable and elegantly furnished, our one b...",Schöneberg,10781,https://www.immobilienscout24.de/expose/138320888,NaN,NaN,NaN,183.000000


Here we see very niche offers.
Small but very comfortable rooms with good furniture.  
For example:
|`Our 19-23 sqm suites for stays over 28 nights are the ideal choice if you are looking for a suitable apartment for two and have therefore been furnished to our highest modern standards. The suites have a fully equipped kitchen, a comfortable box spring bed (1.60 m) with a modern smart TV and a private bathroom with a shower so you can feel at home. If there is dirty laundry, you have the opportunity to wash your clothes in the communal laundry room (opening hours: 6 a.m. to 10 p.m.). Your apartment offers everything you need for a longer stay with us in just one room.` |
|:---| 

It might be an alternative for staying at a hotel.  
But prices here are over 150 euros for sq. meter per month!  
Very high.

#### Warm price


In [ ]:
df[~df.warm_price.isna()][['warm_price', 'cold_price']].describe().T

,count,mean,std,min,25%,50%,75%,max
warm_price,3458.0,1822.842163,1284.870483,250.0,1000.00,1600.0,2290.0,19850.0
cold_price,3458.0,1659.453979,1219.937012,180.0,827.25,1500.0,2100.0,17000.0


3/4 of all listings (where both prices are mentioned) have a cold price less than 2100 and warm price less than 2290 euros.  
Here we can roughly estimate median costs as 100 and q3 as 190.


Let's add a new feature named 'costs' as a substraction of warm and cold prices and do checking.

In [ ]:
df['costs'] = df.warm_price  - df.cold_price
ind = df[df.costs < 0].index
df.loc[ind]

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs
484,132716077,Furnished 2 rooms apartment in Mitte (Berlin),,65.0,2.0,1,1,Personal elevator Person,No garage,5,6,2020.0,Unknown,355,included in additional costs,3091.0,2736.0,1000 + Admin. Fee,Flat,Ukio Germany Gmbh,Frau Julia Morgan,Berlin,"Am Köllnischen Park 17,","Where Berlin conception meets Saharan design, ...",Mitte (Ortsteil),10179,https://www.immobilienscout24.de/expose/132716077,0.0,355.0,0.0,47.553844,-355.0
2690,138189345,Co-Living - THE HOUSE OF CO - Erstbezug Apartment,immediately,27.0,1.0,1,1,Keller cellar elevator recoverer.,1 Underground parking space,1,5,2019.0,Unknown,100,included in additional costs,1149.0,1049.0,2 Kaltmieten,Flat,FU.Life Service GmbH,Booking House of Co,Berlin,"Heidestraße 20,",We combine the best of two living concepts at ...,Moabit,10557,https://www.immobilienscout24.de/expose/138189345,0.0,100.0,0.0,42.555557,-100.0


Warm price is lower than cold price in two these listings.  
It might be an attribute mismatch.
Let's fix this.

In [ ]:
# run this cell only ONCE to fix the error in the data
df.loc[ind, 'warm_price'], df.loc[ind, 'cold_price'] = df.loc[ind, 'cold_price'], df.loc[ind, 'warm_price']
df.loc[ind, 'costs'] = df.loc[ind, 'warm_price'] - df.loc[ind, 'cold_price']
df.loc[ind]

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs
484,132716077,Furnished 2 rooms apartment in Mitte (Berlin),,65.0,2.0,1,1,Personal elevator Person,No garage,5,6,2020.0,Unknown,355,included in additional costs,2736.0,3091.0,1000 + Admin. Fee,Flat,Ukio Germany Gmbh,Frau Julia Morgan,Berlin,"Am Köllnischen Park 17,","Where Berlin conception meets Saharan design, ...",Mitte (Ortsteil),10179,https://www.immobilienscout24.de/expose/132716077,0.0,355.0,0.0,47.553844,355.0
2690,138189345,Co-Living - THE HOUSE OF CO - Erstbezug Apartment,immediately,27.0,1.0,1,1,Keller cellar elevator recoverer.,1 Underground parking space,1,5,2019.0,Unknown,100,included in additional costs,1049.0,1149.0,2 Kaltmieten,Flat,FU.Life Service GmbH,Booking House of Co,Berlin,"Heidestraße 20,",We combine the best of two living concepts at ...,Moabit,10557,https://www.immobilienscout24.de/expose/138189345,0.0,100.0,0.0,42.555557,100.0


In [ ]:
df.warm_price.isna().sum()    # check if there are any NaNs

613

#### `num_rooms`


In [ ]:
df.num_rooms = df.num_rooms.apply(fix_numeric_inconstancies).astype('float16')
df.num_rooms.unique()

array([ 7. ,  2. ,  1. ,  4. ,  3. ,  1.5,  3.5,  5. ,  2.5,  8. ,  5.5,
        6. ,  4.5,  7.5, 11. ], dtype=float16)

We have 1,5, 4,5 rooms, 5,5 rooms, 7,5 rooms, etc.  
It is not a mistake.  
These are the numbers indicated in real advertisements.

In [ ]:
cols = ['num_rooms']
fig = px.box(df[cols], notched=True,  title='Number of rooms',color='variable', width=600)
fig.update_yaxes(matches= None)
fig.update_traces(showlegend=False)
fig.update_layout(xaxis_title="", yaxis_title="Value range")

#### `num_bedrooms`, `num_bathrooms`

In [ ]:
df.num_bathrooms.fillna('?', inplace=True)
df.num_bedrooms.fillna('?', inplace=True)

fig = px.histogram(df[['num_bedrooms','num_bathrooms']], title='Number of bedrooms and bathrooms', color_discrete_sequence=['#1f77b4', '#ff7f0e'],
             labels={'value': 'Number of rooms', 'variable': 'Room type'}, barmode="group", opacity=.7, text_auto= True)
fig.update_layout(xaxis_title="", yaxis_title="Count of properties for rent")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

#### `floor`, `floors_in_building`

Let's save these attributes as categorical variables as we have too many '0' and missing floor numbers.

Let's try to fill missing floor numbers.  
I suggest two approaches, Simple and advanced.  

Simple:   
Filling the floor number for "Ground floor apartment" and 'Basement'  


In [ ]:
# if property is a basement or ground floor apartment, floor is 0 (if floor is not specified)
ind = df[df.floor.isna()  & ((df.property_type == 'Ground floor apartment') | (df.property_type == 'Basement')) ].index
df.loc[ind, 'floor'] = '0'
print(f'Filled {len(ind)} missing floor numbers')

Filled 0 missing floor numbers


Advanced search:  
* look through description for '... located on the ... floor...'
* convert ordinals to digits and paste into the `floor` column

In [ ]:
def extract_floor(string):
  '''
  Extract the floor number from the string
  '''
  if string is None:
    return None
  
  res = re.findall(r'located on the(.{1,20})floor.*', string) # extract the string between 'located on the' and 'floor'
  
  if len(res) > 0:                    # if there is a match        
    res2 = re.findall(r'\d+', res[0]) # look for the digit from in the string

    if len(res2) > 0:                 # if there is a digit in the string
      return str(int(res2[0]))        # return the digit as a string  02 -> 2
    
    else:                             # if there is no digit in the string
      return ordinal_to_digit(re.findall(r'\w+',res[0])[0] ) # convert the ordinal to a digit
  
def ordinal_to_digit(string):
  '''Extract the digit from the ordinal string'''

  # Define a dictionary mapping ordinal strings to integer values
  ordinal_map = {'first': '1', 'second': '2', 'third': '3', 'fourth': '4', 'fifth': '5',
                  'sixth': '6', 'seventh': '7', 'eighth': '8', 'ninth': '9', 'tenth': '10', 'ground': '0',
                  'twelfth': '12', 'thirteenth': '13', 'fourteenth': '14', 'fifteenth': '15', 'sixteenth': '16'}

  # Convert an ordinal string to an integer using the dictionary
  if string in ordinal_map:

      return ordinal_map[string]
  else:
      # print (f'Ordinal {string} not found in the dictionary')
      return string

In [ ]:
floor_ser = df[df.floor.isna() & df.description.str.contains(r'located on the.*floor')]['description']
floor_ser = floor_ser.apply(lambda x: extract_floor(x))
print(f'Filled {floor_ser.value_counts().sum()} floor numbers from the description')
df['floor'] = df.floor.fillna(floor_ser)

Filled 0 floor numbers from the description


We managed to fill 200 (appr. 2 %) missing floor numbers.  
Not a great deal but we tried. 

In [ ]:
df.floor.fillna('?', inplace=True)

In [ ]:
df.floor.fillna('?', inplace=True)
# df.floor = df.floor.astype('float16')
df.floors_in_building.fillna('?', inplace=True)
# df.floors_in_building = df.floors_in_building.astype('float16')

#### `heat costs`, `extra costs`

In [ ]:
def fill_costs (string):
  if string == 'included in additional costs':
    return 0            # if costs are included in additional costs, they are 0
  elif string == 'not specified' or string == 'not included in additional costs':
    return np.nan
  else:
    return float(fix_numeric_inconstancies(string)) # if costs are specified, we fix inconstancies and convert them to float 

In [ ]:
df['heat_costs_calc'] = df.heat_costs.apply (lambda row: fill_costs (row)) 
df['add_costs_calc'] = df.add_costs.apply (lambda row: fill_costs (row)) 

Let's add new column to explore the heat costs more intuitevely.

In [ ]:
df['rel_heat_costs'] = df.heat_costs_calc / df.property_area # relative costs  (EUR/m2)

In [ ]:
fig = px.histogram(df.sort_values(["constr_year"]), x='constr_year',  title='Construction year', text_auto= True) #color='property_type', 
fig.update_layout(xaxis_type = 'category')
# fig.update_layout(xaxis={'categoryorder':'total ascending'})
# fig.update_yaxes(type="log")
fig.update_layout(xaxis_title="", yaxis_title="Count of properties for rent")
fig.show()

In recent years the number of new properties has been increasing.  
Many invest in the construction of residential real estate with a view to subsequent rental.

However, there are too many typos in constr_year and   
most listings do not designate the year of construction at all.
I use the following logic for filling missing values:  
* define a median among specified  
* fill missing values and typos with decreased median (as there is tendency not to mention old constr_year)
* divide the results into age groups

In [ ]:
df['year_group'] = df.constr_year # create a copy of the column

med = df[df.year_group > 0].year_group.median() - 5               # emperically chosen value
print(f'The median of construction year among specified is {med+5}')    
# and replace  with the median
df.loc[df[(df.year_group > 2023) | df.year_group.isna() | (df.year_group == 0) ].index,'year_group'] = med

df['year_group'] = pd.cut(df.year_group, bins=[0, 1900, 1945, 1965, 2000, med, 2020, 2100], labels=['Historic','First half XX cent','Postwar boom','Late XX cent','2000-2014','modern','new'])
df.year_group.value_counts()

The median of construction year among specified is 2014.0


2000-2014             3114
new                    324
modern                 186
Late XX cent           156
Historic               124
First half XX cent     124
Postwar boom            43
Name: year_group, dtype: int64

But we filled  missing values with decreased median to separate the two groups (the fact that owner doesn't mention const_year has it's own value).

### Outliers  
Now we are ready to do some visualizations of data distribution


We'll try to predict these missing warm prices later in p.2.

#### `costs`
Let's look at costs distribution:

In [54]:
px.box(df, x='costs', height= 300, title='Costs per month')

In [55]:
f'{df[df.costs < 5].costs.count() / df.shape[0]:.2%}'

'36.50%'

Many listings (36 %) have equal cold and warm prices, it means heat costs are included in cold price.  
The median value of costs is 120 eur.`

#### `heat_costs`, `add_costs`
This features have a free text format.  
Extracted information is stored in a copies with suffix 'calc' in the names.  

Let's look closer at offerings with extreme costs.


In [56]:
df[~(df.heat_costs.isna() | df.heat_costs.isna())][['heat_costs_calc', 'add_costs_calc']].describe().T.style.format("{:.0f}")

,count,mean,std,min,25%,50%,75%,max
heat_costs_calc,1122,48,88,0,0,0,78,1200
add_costs_calc,1939,255,202,0,130,200,320,3000


In [57]:
px.box(df[['heat_costs_calc', 'add_costs_calc']],  height= 300, orientation='h')

In [58]:
df[df.add_costs_calc > 900].sort_values(by='add_costs_calc', ascending=False).head(5)

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs
1883,133480215,Furnished luxury apartment in the heart of Gru...,01.06.22,197.0,4.0,2,3,Balcony/ terrace balcony/ terrace basement bas...,No garage,1,3,2009.0,Unknown,3.000,not specified,8500.000000,11500.000000,NaN,Flat,Engel & Völkers Immobilien Deutschland GmbH,Engel & Völkers Immobilien Deutschland GmbH,Berlin,,"The villa ""Grunewaldherz"" is a modern building...",Schmargendorf,14195,https://www.immobilienscout24.de/expose/133480215,NaN,3000.00,NaN,43.147209,3000.000000
1908,138488871,Exceptional Living in Jägerstraße on Friedrich...,Nach Absprache,706.0,7.0,4,3,Balcony/ terrace balcony/ terrace basement bas...,Underground parking space,6,6,2007.0,C,2.850,included in additional costs,17000.000000,19850.000000,NaN,Penthouse,CITY-CONCEPT Gesellschaft für Immobilienmanage...,Herr Stefan Schepers,Berlin,"Jägerstraße 34,",The Jägerstraße 34/35 residential and commerci...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/138488871,0.0,2850.00,0.0,24.079321,2850.000000
305,140099183,Life in the Monbijou residence - stately penth...,,375.0,8.0,4,4,Balcony/ terrace balcony/ terrace basement bas...,1 parking space,?,?,1906.0,D,1.724,included in additional costs,9850.000000,11574.000000,"29.403,00 EUR",Other,Engel & Völkers Berlin Mitte GmbH,Engel & Völkers Berlin Mitte,Berlin,"Monbijoustraße 3/5,",This stately maisonett apartment has eight roo...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/140099183,0.0,1724.00,0.0,26.266666,1724.000000
4049,105850244,5-room apartment with a large terrace in the h...,from Juli 2023,343.5,5.0,4,4,Balcony/ terrace balcony/ terrace Passenger el...,1 Underground parking space,3,7,2015.0,Unknown,"1.545,35",included in additional costs,7081.870117,8627.219727,3 Nettokaltmieten,Terrace apartment,HGHI Immobilien Verwaltung GmbH,Frau Marie-Josephine Wahn,Berlin,"Leipziger Str. 12,",As a unique residential area over the roofs of...,Mitte (Ortsteil),10117,https://www.immobilienscout24.de/expose/105850244,0.0,1545.35,0.0,20.616798,1545.349609
3848,141147504,Fantastic view of the zoo,Nach Absprache,152.5,3.0,2,2,Balcony/ terrace balcony/ terrace basement bas...,1 Underground parking space,5,9,2003.0,C,1.350,included in additional costs,2898.000000,4248.000000,8694,Flat,FRASSEK Private Real Estate GmbH,Herr Michael Frassek,Berlin,,As special as the location on Potsdamer Platz ...,Tiergarten,10117,https://www.immobilienscout24.de/expose/141147504,0.0,1350.00,0.0,19.003279,1350.000000


Exclusive offerings with parking places and heat costs included to extra costs.  

For recall, original costs strings are stored separately in `heat_costs` and `add_costs`. 

#### `deposit`

`deposit` incude information about required deposit.  
Some ads include specific  value, others - the number of monthly paid cold prices.   
Let's clean this data:
1. retrive only digits
2. if value < 13 (common practice 3) then multiply by cold price

In [59]:
df.deposit.fillna('0', inplace=True) # we assume that if deposit is not specified, there owner doesn't require a deposit

We fill missing values with zeros (0 means that the owner does not require a deposit at all),  
however,   3 months deposit is a standard in Germany.

In [60]:
def format_deposit(row):
  '''
  Reformat  deposit column
  '''
  # if deposit is given in words, convert it to n- months amount of cold_price
  
  if 'drei' in str.lower(row['deposit']):
    return 3 * row['cold_price']
  elif 'zwei' in str.lower(row['deposit']):
    return 2 * row['cold_price']
  elif 'ein' in str.lower(row['deposit']):
    return row['cold_price']

  res = fix_numeric_inconstancies(row['deposit'])

  if  res is None:    # the owner really doesn't require a deposit
    # print('the owner really doesn"t require a deposit')
    return 0

  if res == 0:
    return 0
  elif res < 13: 
    return res * row['cold_price'] # if deposit is given in months, convert it to EUR
  else:
    return res

Processing deposit column

In [61]:
ind = df[df.deposit.apply(lambda x: 'drei' in str.lower(x))].index # 3 months
df['deposit_calc'] = df.apply(lambda row: format_deposit(row), axis=1)

df.loc[ind,].head(3)

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs,deposit_calc
458,140224410,New building in the first cover - sample apart...,01.05.2023,111.9375,5.0,NaN,NaN,Balcony/ terrace balcony/ terrace Passenger el...,No garage,1,5,2023.0,B,"229,46","254,08",1287.199951,1770.739990,drei Nettokaltmieten,Unknown,degewo,degewo Köpenicker Wohnungsgesellschaft mbH,Berlin,"Igelsteig 7B,",All apartments in this new building are equipp...,Köpenick,12557,https://www.immobilienscout24.de/expose/140224410,254.08,229.46,2.269838,11.499273,483.540039,3861.599854
518,141383819,* First cover in the new building near the Müg...,from immediately,107.7500,5.0,NaN,NaN,Personal elevator Personal Guest toilet guest ...,No garage,1,3,2023.0,A,"219,87","286,69",1023.909973,1530.469971,drei Nettokaltmieten,Unknown,degewo,degewo AG,Berlin,"Fürstenwalder Allee 324,",A total of 386 apartments in 34 buildings for ...,Rahnsdorf,12589,https://www.immobilienscout24.de/expose/141383819,286.69,219.87,2.660696,9.502645,506.559998,3071.729919
519,141383671,* First cover in the new building near the Müg...,from immediately,85.8125,3.0,NaN,NaN,Personal elevator person elevator,No garage,3,3,2023.0,A,"175,01","228,20",986.590027,1389.800049,drei Nettokaltmieten,Unknown,degewo,degewo AG,Berlin,"Fürstenwalder Allee 326,",A total of 386 apartments in 34 buildings for ...,Rahnsdorf,12589,https://www.immobilienscout24.de/expose/141383671,228.20,175.01,2.659286,11.497044,403.210022,2959.770081


We calculated the sum of the deposit (deposit_calc) based on free form text information (deposit).  
In the example above the deposit was calculated on 'drei Nettokaltmieten' information.  
Let's check the distribution:

In [62]:
px.box(df[['deposit_calc']], x='deposit_calc', notched=True, title='Deposit ')

The median is 1500 and 75% of listings have a deposit less than 3150 euros.  
And we have a big outliers:

In [63]:
df[df.deposit_calc/df.cold_price > 5].sort_values(by='deposit_calc', ascending=False).head(5)

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs,deposit_calc
2228,141083728,Only with WBS 100/140-family apartment with ba...,01.05.2023,86.625,4.0,3,1,Balcony/ terrace balcony/ terrace basement bas...,No garage,2,?,1987.0,Unknown,"173,28","105,70",626.020020,905.000000,"187.080,60 €",Flat,Immonexxt GmbH,Frau Stefanie Hammer,Berlin,"Dora-Mendler-Straße 3,",The houses created in the mid -1970s are in a ...,Rudow,12355,https://www.immobilienscout24.de/expose/141083728,105.7,173.28,1.220202,7.226782,278.979980,187080.600000
1845,138737107,Oppress apartments with the expansion quality ...,1.3.2023,136.125,3.0,NaN,2,Balcony/ Terrace Balcony/ Terrace Passenger Ri...,No garage,5,5,1890.0,C,"585,23",included in additional costs,3674.699951,4259.930176,11,Attic,Private,Frau Forsting Christiane,Berlin,,First cover.Top equipment according to individ...,Schöneberg,10827,https://www.immobilienscout24.de/expose/138737107,0.0,585.23,0.000000,26.995041,585.230225,40421.699463


Here we can see two cases:
1. cold_price '626' and deposit '187.080,60 €' (3x cold_price = 1878). It's a typo.
2. cold_price '3675' and deposit '11'. I checked this listing. Let's leave as it is. (11 month deposit)
Let's fix it.

In [64]:
ind = df[df.property_id == 141083728].index
df.loc[ind, 'deposit_calc'] = df.loc[ind, 'cold_price'] * 3
df.loc[ind]

,property_id,title,logging_date,property_area,num_rooms,num_bedrooms,num_bathrooms,criteria,garage,floor,floors_in_building,constr_year,energy_eff,add_costs,heat_costs,cold_price,warm_price,deposit,property_type,publisher,contact,city,address,description,region,zip,link,heat_costs_calc,add_costs_calc,rel_heat_costs,cold_price_rel,costs,deposit_calc
2228,141083728,Only with WBS 100/140-family apartment with ba...,01.05.2023,86.625,4.0,3,1,Balcony/ terrace balcony/ terrace basement bas...,No garage,2,?,1987.0,Unknown,"173,28","105,70",626.02002,905.0,"187.080,60 €",Flat,Immonexxt GmbH,Frau Stefanie Hammer,Berlin,"Dora-Mendler-Straße 3,",The houses created in the mid -1970s are in a ...,Rudow,12355,https://www.immobilienscout24.de/expose/141083728,105.7,173.28,1.220202,7.226782,278.97998,1878.060059


In [65]:
df[df.deposit_calc.isna()]['deposit']

3826        Germany
4024    Deutschland
Name: deposit, dtype: object

In two ads, all of Germany was required as a deposit.  
Apparently these properties are very valuable for their owners.  
But we have to fix it  ;)

In [66]:
ind = df[df.deposit_calc.isna()].index
df.loc[ind, 'deposit_calc'] = 3 * df.loc[ind, 'cold_price']

#### `criteria`

The idea is to define the most popular criteries and refill criteria desciptions:
1. form a bag of words (criteria)  
1. remove all stop words as some descriptions are not concise and in a free form.  
2. leave only those criteries that are most popular.  

Let's code:  
First, we fill the missing values:

In [75]:
df.criteria.fillna("", inplace=True)

In [76]:
def get_unique_values(string):
  '''
  Get unique values for each criteria. 
  '''
  inp = string.lower()
  inp = inp.replace('/','').split(' ')  # remove slashes and split by space
  if len(string) < 180:                 # if the string is too long, it's probably a mistake
     
    crit= []
    for word in inp:                    # get unique values         
      if word not in crit:
          crit.append(word)

    return " ".join(crit)               # return a string of unique values separated by space
  else:
     return ""                          # return empty string       

This is an example of contamination in criteria column:

In [77]:
df[df.criteria.apply(lambda x: len(x)) > 180].criteria.head(3)

12    Online tour possible online tour online tour o...
16    Online tour possible online tour online tour o...
43    Online tour possible online tour online tour o...
Name: criteria, dtype: object

In [78]:
df['criteria_clean'] = df.criteria.apply(lambda x: get_unique_values(x))  # create a new column with cleaned criteria

text = " ".join(df.criteria_clean.unique()) # get ALL the criteria (in the data set) in one string
words = text.split()                        # split the string into words

word_counts = {}                            # create a dictionary to store the word counts              
for word in words:                          # iterate over the words
    if word in word_counts:                 # if the word is already in the dictionary, increment the count
        word_counts[word] += 1
    else:
        word_counts[word] = 1               # we have a new word, add it with a count of 1

stop_words = set(stopwords.words('english')) # get the list of stop words from NLTK

# Delete the keys that are stop words
for word in stop_words:                      # iterate over the stop words
    if word in word_counts:                  # if the stop word is in the dictionary, delete it
        del word_counts[word]                # delete the stop word from the dictionary

# Filter out the rare words
crit_dict = {word: count for word, count in word_counts.items() if count > 10}.keys() # get the words that occur more than 10 times 
crit_dict 

dict_keys(['balcony', 'terrace', 'basement', 'passenger', 'elevator', 'personal', 'kitchen', 'fitted', 'guest', 'toilet', 'keller', 'garden', 'use', 'stepless', 'access', 'mind', 'built', '-in', 'built-in'])

And finally filter the criteria with prevalence:

In [79]:
def filter_criteria(string, criteria_dict):
    '''Filter the criteria to keep only the words that are in the criteria dictionary'''
    return ' '.join(word for word in string.split() if word in criteria_dict)

df['criteria_clean'] = df['criteria_clean'].apply(lambda x: filter_criteria(x, crit_dict))

#### `logging_date`  
is not a neccessary parameter.  
Some rows include availiable date to log, others additional notes.  

I replace nans here  with `""`.

In [ ]:
df.logging_date.fillna("", inplace=True)

#### `garage`, `energy_eff`

In [ ]:
df.garage.fillna("No garage", inplace=True)      # we assume that if garage is not specified, there is no garage
df.energy_eff.fillna("Unknown", inplace=True)    # we assume that if energy efficiency is not specified, it has no EE certificate 
df.property_type .fillna("Unknown", inplace=True)

#### `address`, `region`, `zip`

In [ ]:
df.address.fillna('', inplace=True)
df.region.fillna('', inplace=True)
df.zip.fillna('', inplace=True)

We still have nan values in `warm_price` and associated with it `costs`.  

In order to address this issue more advanced techique required.
We will deal with it in the next part

### Save proccessed data 

In [85]:
# df.to_csv(path_to_csv + 'Berlin_housing_proccessed.csv', sep=';', index=False)